## 简答题

1. TensorFlow 是否可以简单替代 NumPy？两者之间的主要区别是什么？
不能简单替代；NumPy 仅 CPU、无自动求导，TF 支持 GPU/TPU、可导、图优化
2. 使用 `tf.range(10)` 和 `tf.constant(np.arange(10))` 是否会得到相同的结果？
数值相同，但数据类型可能不同
3. 可以通过编写函数或继承 `tf.keras.losses.Loss` 来定义自定义损失函数。两种方法分别应该在什么时候使用？
简单无状态损失用函数，需要参数/序列化/有状态时用继承 Loss 类
4. 可以直接在函数中定义自定义指标或采用 `tf.keras.metrics.Metric` 子类。两种方法分别应该在什么时候使用？
函数：单批即得结果；子类：需跨批累积
5. 什么时候应该自定义层而不是自定义模型？
定义可复用计算块时自定义层，定义完整网络架构或训练行为时自定义模型
6. 有哪些示例需要编写自定义训练循环？
GANs等对抗训练、强化学习、自定义梯度处理、多模型联合训练等复杂场景
7. 自定义 Keras 组件中可以包含任意 Python 代码，还是必须转换为 TF 函数？
必须转换为 TF 函数。任意 Python 代码可能导致图构建失败或性能下降
8. 如果要将函数转换为 TF 函数，应避免哪些主要模式？
避免使用 Python方法、内部创建变量、使用可变数据结构输入、依赖会改变的全局变量。
9. 何时需要创建动态 Keras 模型？ 如何动态创建Keras模型？为什么不是所有模型都动态化？
当模型结构需在运行时确定时（如 RNNs、树状网络）。通过继承 Model 类并在 call 方法中使用控制流创建。不都动态化是因为静态图性能更好、更易部署和可视化。

## 编程题

1. 实现一个执行层归一化的自定义层：
    - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
    - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
    - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：

    - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
    - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。

In [1]:
import tensorflow as tf

class LayerNorm(tf.keras.layers.Layer):
    def __init__(self, eps=1e-3, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps

    def build(self, input_shape):
        dim = input_shape[-1]
        self.alpha = self.add_weight(
            name='alpha', shape=(dim,), dtype=tf.float32,
            initializer='ones', trainable=True)
        self.beta = self.add_weight(
            name='beta', shape=(dim,), dtype=tf.float32,
            initializer='zeros', trainable=True)

    def call(self, inputs, training=None):
        mean, var = tf.nn.moments(inputs, axes=-1, keepdims=True)
        std = tf.sqrt(var + self.eps)
        return self.alpha * (inputs - mean) / std + self.beta

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# 数据
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_val = x_train/255., x_val/255.

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
            .shuffle(60000).batch(128))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(128)

# 模型（用刚才的 LayerNorm）
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128),
    LayerNorm(),        # 自定义层
    layers.Activation('relu'),
    layers.Dense(64),
    LayerNorm(),
    layers.Activation('relu'),
    layers.Dense(10)
])

# 不同层不同优化器/学习率
opt1 = tf.keras.optimizers.Adam(1e-3)      # 前面层
opt2 = tf.keras.optimizers.SGD(1e-1)       # 最后层

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
val_loss = tf.keras.metrics.Mean()
val_acc = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss = loss_fn(y, logits)
    # 分层取变量
    vars1 = model.layers[2].trainable_variables + model.layers[0].trainable_variables # alpha beta
    vars2 = model.layers[-1].trainable_variables # kernel b
    grads = tape.gradient(loss, vars1 + vars2)
    grads1, grads2 = grads[:len(vars1)], grads[len(vars1):]
    opt1.apply_gradients(zip(grads1, vars1))
    opt2.apply_gradients(zip(grads2, vars2))
    train_loss.update_state(loss)
    train_acc.update_state(y, logits)

@tf.function
def val_step(x, y):
    logits = model(x, training=False)
    val_loss.update_state(loss_fn(y, logits))
    val_acc.update_state(y, logits)

EPOCHS = 5
for epoch in range(1, EPOCHS+1):
    train_loss.reset_states(); train_acc.reset_states()
    val_loss.reset_states(); val_acc.reset_states()
    for it, (x, y) in enumerate(train_ds, 1):
        train_step(x, y)
        print(f'E{epoch} Iter{it}  loss={train_loss.result():.4f}  acc={train_acc.result():.4f}', end='\r')
    for x, y in val_ds:
        val_step(x, y)
    print(f'E{epoch}  train_loss={train_loss.result():.4f}  train_acc={train_acc.result():.4f}  '
          f'val_loss={val_loss.result():.4f}  val_acc={val_acc.result():.4f}')

E1  train_loss=1.0711  train_acc=0.6472  val_loss=0.8191  val_acc=0.7030
E2  train_loss=0.7469  train_acc=0.7355  val_loss=0.7300  val_acc=0.7329
E3  train_loss=0.6889  train_acc=0.7531  val_loss=0.6913  val_acc=0.7480
E4  train_loss=0.6591  train_acc=0.7625  val_loss=0.6693  val_acc=0.7578
E5  train_loss=0.6387  train_acc=0.7695  val_loss=0.6526  val_acc=0.7649
